In [1]:
import pandas as pd
import numpy as np

# creamos los dos DataFrame, uno con los datos de las noticias y otro con la información sobre los municipios

df = pd.read_csv("Data\Meneos_Iniciales_y_finales_2023.csv")
df_municipios = pd.read_csv("Data/MUNICIPIOS.csv", sep = ";", encoding = "iso-8859-2")

# definimos una función para que nos retorne el primer municipio que encuentre

def funcion_municipios(x):
    lista_municipios = df_municipios["NOMBRE_ACTUAL"].tolist()
    encontrado = False
    for municipio in lista_municipios:
        if municipio in x.split():
            encontrado = True
            municipio_encontrado = municipio
            break

    if encontrado:
        return municipio_encontrado
    else:
        return False

# aplicamos la función a la columna Titular 

df["Municipio"] = df["Titular"].apply(funcion_municipios)


# aplicamos la función a la columna Titular, solo en aquellos valores de la columna Municipio que no contengan ya un municipio

indices = df[df["Municipio"] == False].index.to_list()
for i in indices:
    df.at[i, "Municipio"] = funcion_municipios(df.loc[i, "Entradilla"])

# hacemos un merge para obtener las columnas Provincia, Latitud y Longitud correspondientes a cada municipio:

df = df.merge(df_municipios[["PROVINCIA","LONGITUD", "LATITUD","NOMBRE_ACTUAL"]], 
              left_on = "Municipio", right_on = "NOMBRE_ACTUAL", how = "left").drop("NOMBRE_ACTUAL", axis=1)

# ta-dá
df

C:\Users\deepi\AppData\Local\Temp\ipykernel_8796\2966529455.py:39: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dict_geo = df_municipios.loc[:, ["NOMBRE_ACTUAL", "PROVINCIA", "LONGITUD", "LATITUD"]].set_index("NOMBRE_ACTUAL").T.to_dict('list')


,Indice,Comunidad,Usuario,Medio,Enviado,Publicado,Titular,Entradilla,Votos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma,Municipio,Provincia,Latitud,Longitud
0,0,politica,dmeijide,twitter.com,31/12/2023 12:05,31/12/2023 22:40,Yanis Varoufakis: Suspended el Apartheid Israe...,Como una vez suspendimos el Apartheid de Sudáf...,649,1237,276,373,9,57,399,False,False,False,False
1,1,actualidad,--557077--,europapress.es,31/12/2023 17:11,31/12/2023 21:25,Irán libera al español Santiago Sánchez Cogedo...,La Justicia de Irán ha puesto en libertad este...,280,1206,140,140,1,108,499,False,False,False,False
2,2,actualidad,tropezon,diariodecadiz.es,30/12/2023 23:31,31/12/2023 21:05,Estafan a un vecino de Vejer casi 10.000 euros...,Francisco Raúl Moreno Mera recibió tres mensaj...,236,4343,103,133,2,110,473,False,False,False,False
3,3,actualidad,FatherKarras,kaosenlared.net,31/12/2023 17:13,31/12/2023 20:25,Pablo González: un secuestro político a la com...,"Cuesta entender, incluso si le añadimos la eti...",573,710,232,341,17,124,486,False,False,False,False
4,4,actualidad,silver_hoodlum,twitter.com,31/12/2023 19:12,31/12/2023 19:40,"Video: El cacheo pacífico de los soldados ""Isr...",Un video vale mas que mil palabras de Netanyah...,580,4640,235,345,5,77,463,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,tecnología,--682766--,sincrogo.com,01/01/2023 00:34,01/01/2023 03:25,Cambios en la IT: Los trabajadores ya no tendr...,¡Importante!: Los trabajadores ya no tendrán q...,260,3963,123,137,1,57,509,False,False,False,False
332,332,ocio,Andaui,letraslibres.com,31/12/2022 21:05,01/01/2023 02:30,El gol imposible de Pelé,"Gracias a Pelé, un jugador alegre, juvenil y l...",82,5383,40,42,7,38,427,False,False,False,False
333,333,tecnología,ccguy,xatakamovil.com,01/01/2023 00:35,01/01/2023 01:25,"Por increíble que parezca, mi aplicación favor...",Si hay una app que me ha gustado y se ha conve...,310,9966,143,167,9,101,477,False,False,False,False
334,334,ciencia,robustiano,francis.naukas.com,31/12/2022 22:36,01/01/2023 00:55,Nuevo récord de factorización de números enter...,El algoritmo de Shor para factorizar números e...,142,1705,73,69,1,14,490,False,False,False,False


In [133]:
# NO VALIDO
df[df["Municipio"] == False]["Municipio"] = df[df["Municipio"] == False]["Entradilla"].apply(funcion_municipios)

# creamos un diccionario que contenga como llaves los nombres de los municipios y como valores una lista con la provincia, la latitud y la longitud de cada municipio

dict_geo = df_municipios.loc[:, ["NOMBRE_ACTUAL", "PROVINCIA", "LONGITUD", "LATITUD"]].set_index("NOMBRE_ACTUAL").T.to_dict('list')

# creamos las columnas Provincia, Latitud y Longitud usando la información contenida en nuestro diccionario

df["Provincia"] = df["Municipio"].map(lambda x : dict_geo[x][0] if x != False else x)
df["Latitud"] = df["Municipio"].map(lambda x : dict_geo[x][2] if x != False else x)
df["Longitud"] = df["Municipio"].map(lambda x : dict_geo[x][1] if x != False else x)

C:\Users\deepi\AppData\Local\Temp\ipykernel_24068\3393339837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["Municipio"] == False]["Municipio"] = df[df["Municipio"] == False]["Entradilla"].apply(funcion_municipios)
